# Basic RAG over an existing vector database

In [1]:
!pip install pdfplumber
!pip install langchain langchain_ollama
!pip install chromadb sentence-transformers langchain_huggingface langchain_chroma

# Librerias

In [2]:
import os
import re
import pdfplumber
import requests
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Obtener los documentos de un pdf

In [3]:

# URL del archivo PDF que quieres descargar
url = 'https://www.esferalibros.com/wp-content/uploads/2022/02/Jugones-de-la-NBA-primeras.pdf'
url2 = 'https://www.lectio.es/tasts/100porcienNBATAST.pdf'

# Nombre con el que deseas guardar el archivo localmente
output_file = 'archivo_descargado_jugones.pdf'

output_file2 = 'archivo_descargado_NBATAST.pdf'

try:
    # Realizar la solicitud GET
    response = requests.get(url, stream=True)
    
    # Verificar que la solicitud fue exitosa
    response.raise_for_status()  # Lanza una excepción si el estado HTTP no es 200
    
    # Guardar el contenido en un archivo local
    with open(output_file, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):  # Descargar en bloques de 8 KB
            file.write(chunk)
    
    print(f"Archivo descargado y guardado como '{output_file}'")
except requests.exceptions.RequestException as e:
    print(f"Error al descargar el archivo: {e}")

try:
    # Realizar la solicitud GET
    response = requests.get(url2, stream=True)
    
    # Verificar que la solicitud fue exitosa
    response.raise_for_status()  # Lanza una excepción si el estado HTTP no es 200
    
    # Guardar el contenido en un archivo local
    with open(output_file2, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):  # Descargar en bloques de 8 KB
            file.write(chunk)
    
    print(f"Archivo descargado y guardado como '{output_file2}'")
except requests.exceptions.RequestException as e:
    print(f"Error al descargar el archivo: {e}")

Archivo descargado y guardado como 'archivo_descargado_jugones.pdf'
Archivo descargado y guardado como 'archivo_descargado_NBATAST.pdf'


In [4]:
# Ruta del archivo PDF
file_path = 'archivo_descargado_jugones.pdf'
file_path2 = 'archivo_descargado_NBATAST.pdf'

lista_nba = []
try:
    with pdfplumber.open(file_path) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text = page.extract_text()
            lista_nba.append(text)
            if text:
                print(f"--- Página {page_number} ---")
                print(text)
            else:
                print(f"La página {page_number} no contiene texto legible.")
except FileNotFoundError:
    print(f"El archivo '{file_path}' no existe.")
except Exception as e:
    print(f"Ocurrió un error al leer el PDF: {e}")

try:
    with pdfplumber.open(file_path2) as pdf:
        for page_number, page in enumerate(pdf.pages, start=1):
            text2 = page.extract_text()
            lista_nba.append(text2)
            if text2:
                print(f"--- Página {page_number} ---")
                print(text2)
            else:
                print(f"La página {page_number} no contiene texto legible.")
except FileNotFoundError:
    print(f"El archivo '{file_path}' no existe.")
except Exception as e:
    print(f"Ocurrió un error al leer el PDF: {e}")


--- Página 1 ---
Javier Serrano
JUG NES DE LA NBA
Conoce la historia y los éxitos
de Jordan, Kobe, LeBron o Gasol
para llegar a ser como ellos
jjuuggoonneessNNBBAA__2211((1133))[[eedduu]]..iinndddd 55 22//22//2222 1100::4488
--- Página 2 ---
Índice
Introducción ........................................................... 9
Michael Jordan .......................................................... 17
LeBron James ............................................................ 21
Kobe Bryant .............................................................. 25
Pau Gasol ................................................................... 29
Stephen Curry ............................................................ 33
Kevin Durant .............................................................. 38
Kareem Abdul-Jabbar ................................................. 43
Hakeem Olajuwon ....................................................... 47
Shaquille O'Neal .........................................

In [5]:
lista_nba

['Javier Serrano\nJUG NES DE LA NBA\nConoce la historia y los éxitos\nde Jordan, Kobe, LeBron o Gasol\npara llegar a ser como ellos\njjuuggoonneessNNBBAA__2211((1133))[[eedduu]]..iinndddd 55 22//22//2222 1100::4488',
 "Índice\nIntroducción ........................................................... 9\nMichael Jordan .......................................................... 17\nLeBron James ............................................................ 21\nKobe Bryant .............................................................. 25\nPau Gasol ................................................................... 29\nStephen Curry ............................................................ 33\nKevin Durant .............................................................. 38\nKareem Abdul-Jabbar ................................................. 43\nHakeem Olajuwon ....................................................... 47\nShaquille O'Neal ......................................................

# Seleccionamos un modelo para las embeddings

In [6]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/Trabajo_BIDA/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Añadimos los datos

Creamos vector storage

In [7]:
if not os.path.exists("./Pdfs"):  # Comprobar se existe o directorio
    vector_store = Chroma.from_texts(
        texts=lista_nba,  # Lista de textos limpos
        collection_name="datos_pdfs",  # Nome da colección
        embedding=embeddings,  # Embeddings configurados
        persist_directory="./Pdfs",  # Directorio para persistir os datos
    )
else:
    vector_store = Chroma(
        collection_name="datos_pdfs",
        embedding_function=embeddings,
        persist_directory="./Pdfs",
    )


Create retriever

In [8]:
# Paso 5: Configurar o retriever
retriever = vector_store.as_retriever(
    search_type="similarity",  # Tipo de búsqueda
    search_kwargs={"k": 3}  # Número de resultados relevantes a devolver
)

In [9]:
# Realizar unha consulta
query = "Jugadores de la NBA"
results = retriever.get_relevant_documents(query)  # Invoca os documentos relevantes

# Mostrar os resultados
for i, result in enumerate(results):
    print(f"Resultado {i+1}:")
    print(result.page_content)  # Usar 'page_content' en lugar de 'get'
    print()


/tmp/ipykernel_6154/571088611.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)  # Invoca os documentos relevantes


Resultado 1:
La NBA
Antes de meternos a hablar de jugadores, hagamos un reco-
rrido rápido para conocer mejor esta competición. La NBA
(National Basketball Association) es una liga profesional de
baloncesto que surgió en Estados Unidos a mediados del
xx
siglo , en 1946.
Desde entonces, los mejores jugadores del mundo han
querido desarrollar allí su carrera, por prestigio, porque ser
una estrella de la NBA es lo máximo a lo que puede aspirar
un baloncestista, y por dinero, es la liga que mejor paga a los
jugadores. Las grandes estrellas actuales ganan casi 50 millo-
nes de dólares al año.
Durante las primeras décadas, prácticamente todos los
jugadores de la Liga eran estadounidenses, y a partir de los años
ochenta, empezaron a llegar con regularidad jugadores de
otros lugares del mundo. Algunos fueron verdaderas estrellas,
como Dirk Nowitzki (Alemania), Hakeem Olajuwon (Nige-
ria), Yao Ming (China), Pau Gasol (España), o jugadores más
recientes como Giannis Antetokounmpo (Grecia), Luka


RAG Chain

In [10]:
# Prompt
template = """Answer the question based only on the following context:
{context} In spanish as long as you can .

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [11]:
# Question
chain.invoke("Jugadores NBA")


'El texto menciona varios jugadores destacados de la NBA, como:\n\n* Michael Jordan\n* Dirk Nowitzki (Alemania)\n* Hakeem Olajuwon (Nigeria)\n* Yao Ming (China)\n* Pau Gasol (España)\n* Giannis Antetokounmpo (Grecia)\n* Luka Doncic (Eslovenia)\n* Nikola Jovic (Serbia)\n\nTambién menciona a algunos de los jugadores que jugaron con Michael Jordan en los Chicago Bulls, como:\n\n* Scottie Pippen\n* Dennis Rodman\n* Horace Grant\n\nEn resumen, el texto presenta una variedad de jugadores destacados de la NBA que han contribuido al deporte a lo largo de los años.'

In [12]:
chain.invoke("Quien es Michael Jordan?")

'Michael Jordan es un jugador de baloncesto legendario. Fue uno de los mejores jugadores de la historia del deporte, conocido por su habilidad en el salto y su éxito en la NBA, así como su carrera exitosa fuera del baloncesto, incluyendo ser campeón olímpico dos veces y ser protagonista de la película "Space Jam".'

In [13]:
chain.invoke("Quien es LeBron James?")

'LeBron James es uno de los mejores jugadores de la historia de la NBA. Nació el 30 de diciembre de 1984 en Akron, Ohio, y mide 2,06 metros de altura. Es un jugador versátil que puede jugar en varias posiciones del campo, como base, pívot o escolta, y es conocido por su habilidad para lanzar canasta desde cualquier posición y su capacidad para pasar el balón con precisión. También es un defensor destacado y ha ganado varios títulos importantes, incluyendo dos medallas de oro olímpicas y una medalla de bronce en el Mundial de 2006.'